In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format ='{:,.3f}'.format

import seaborn as sns
from scipy import stats

import matplotlib.pyplot as plt
from pylab import rcParams
from mpl_toolkits.mplot3d import Axes3D
plt.style.use("fivethirtyeight")

#%matplotlib inline
%config InlineBackend.figure_format = "svg"
rcParams["figure.figsize"] = 5, 3

In [2]:
train_df = pd.read_csv("train.csv")

In [3]:
train_df.head(5)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.000,47.982,29.443,6.000,7,9.000,1969,0.089,B,B,33,7976,5,nan,0,11,B,"184,966.931"
1,15053,41,3.000,65.684,40.050,8.000,7,9.000,1978,0.000,B,B,46,10309,1,240.000,1,16,B,"300,009.450"
2,4765,53,2.000,44.948,29.198,0.000,8,12.000,1968,0.050,B,B,34,7759,0,229.000,1,3,B,"220,925.909"
3,5809,58,2.000,53.353,52.732,9.000,8,17.000,1977,0.438,B,B,23,5735,3,"1,084.000",0,5,B,"175,616.227"
4,10783,99,1.000,39.649,23.776,7.000,11,12.000,1976,0.012,B,B,35,5776,1,"2,078.000",2,4,B,"150,226.532"


In [4]:
test_df = pd.read_csv("test.csv")

In [5]:
test_df.head(5)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.000,49.883,33.433,6.000,6,14.000,1972,0.310,B,B,11,2748,1,nan,0,0,B
1,15856,74,2.000,69.263,nan,1.000,6,1.000,1977,0.076,B,B,6,1437,3,nan,0,2,B
2,5480,190,1.000,13.598,15.948,12.000,2,5.000,1909,0.000,B,B,30,7538,87,"4,702.000",5,5,B
3,15664,47,2.000,73.047,51.941,9.000,22,22.000,2007,0.102,B,B,23,4583,3,nan,3,3,B
4,14275,27,1.000,47.527,43.388,1.000,17,17.000,2017,0.072,B,B,2,629,1,nan,0,0,A


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


Чуть больше 20% значений отсутсвует по фиче LifeSquare и почти половина по Healthcare_1.
Проанализируем каждую переменную построив гистограммы частот по количественным признакам

In [7]:
# fig, ax = plt.subplots(nrows=1, ncols=2)
# ax1, ax2 = ax
# fig.subplots_adjust(wspace=0.4)
# fig.set_size_inches(20, 10)

# ax1.hist(train_df["DistrictId"], bins=20, density=True)
# ax1.set_title("Distribushion of 'DistrictId'", fontdict={"fontsize": 15})
# ax1.set_ylabel("Density", fontdict={"fontsize": 15})
# ax1.set_xlabel("District ID", fontdict={"fontsize": 15})


# ax2.scatter(train_df["DistrictId"], train_df["Price"], s=0.5)
# ax2.set_title("District_ID vs Price", fontdict={"fontsize": 15})
# ax2.set_xlabel("District ID", fontdict={"fontsize": 15})
# ax2.set_ylabel("Price", fontdict={"fontsize": 15})

# fig.savefig("graphs/1._Graph_1.jpg")

Как видно из правого графика основная масса наблюдений приходится на районы с ID до 100. Определенных зависимостей между районом и ценой нет. Районы с ID выше 100 представляют собой скорее всего малонаселенные районы, потому что распределение цен примерно одинаковое что в районах с высокой плотностью, что в районах с низкой плотностью. Так же четко прослеживается определенная граница на уровне 100 000. Скорее всего это представляет собой набор примерно одинаковых лотов....возможно лоты с определенными показателями, типо минимальных по размеру однушек или студий. Оценим уровень наблюдений по районам от 0 до 250 с шагом в 50.

In [8]:
j = 0
for i in range(5):
    lot_count = train_df["DistrictId"].value_counts(sort=False).sort_index()[j:j+50].sum()
    print(f"Number of lots in districts from {j} to {j+50} = {lot_count}")
    j += 50

Number of lots in districts from 0 to 50 = 5872
Number of lots in districts from 50 to 100 = 2680
Number of lots in districts from 100 to 150 = 1086
Number of lots in districts from 150 to 200 = 355
Number of lots in districts from 200 to 250 = 7


Почти 60% лотов сосредоточены в районах от 0 до 50. Скорее всего данное распределение связано с популярностью района и его плотностью населения. Скорее всего в данном случае другие фичи, описывающие социальные показатели так же зависят от конкретного района. Так же можно попробовать разбить данные по ценам на два массива и с помощью т-теста посчитать значимо различаются ли их средние

In [9]:
# fig, ax = plt.subplots(nrows=1, ncols=1)
# fig.set_size_inches(20,10)

# ax.hist(train_df["Price"].to_numpy(), bins=100, density=True)
# ax.set_title("Price distribushion", fontdict={"fontsize": 15})
# ax.set_xlabel("Price", fontdict={"fontsize": 15})
# ax.set_ylabel("Density", fontdict={"fontsize": 15})

# fig.savefig("graphs/2._Graph_2.jpg")

Как видно распределение конечно не нормальное, но по крайней мере не имеет большое количество выбросов и сама форма приемлима. По хорошему тут надо было бы сделать тест на нормальность, но ограничусь пока этим))

In [10]:
price_1 = train_df.loc[train_df["DistrictId"] <= 50, "Price"].to_numpy()
price_2 = train_df.loc[train_df["DistrictId"] > 50, "Price"].to_numpy()

In [11]:
# fig, ax = plt.subplots(nrows=1, ncols=1)
# fig.set_size_inches(20,10)

# ax.hist(price_2, bins=100, density=True, color="b", label="District ID >50")
# ax.hist(price_1, bins=100, density=True, color="g", label="District ID <= 50", alpha=0.7)
# ax.set_title("Price distribushion vs District", fontdict={"fontsize": 15})
# ax.set_xlabel("Price", fontdict={"fontsize": 15})
# ax.set_ylabel("Dencity", fontdict={"fontsize": 15})
# ax.legend(fontsize=20)

# fig.savefig("graphs/3._Graph_3.jpg")

Из графика видно, что распределение кластера 1 (с районами до 50) скошено влево, а второй кластер наоборот. Скорее всего это говорит о том, что в кластере 1 присутсвует большое количество лотов схожих характеристик, которые относятся к масс-маркету. Во стором кластере придставлены районы, где наблюдается превалирование более дорогих лотов.

In [12]:
price_1.var(), price_2.var(), price_2.var() / price_1.var()

(7036977749.638955, 9992983379.90366, 1.4200674970751994)

Дисперсия по второму кластеру (где номер района больше 50) выше на 30%, поэтому применим параметр equal_var

In [13]:
stats.ttest_ind(price_1, price_2, equal_var=False)

Ttest_indResult(statistic=-20.487798035077642, pvalue=6.370376149224799e-91)

Исходя из того что получили минимальное значение р-значимости, можно отвергнуть нулевую гипотезу о равенстве средних.

Начнем проверять распределение цен в зависимсоти от районов и социальных показателей, возможно можно будет найти какие-то взаимосвязи Начнем с показателей Ecology_1 Ecology_2 Ecology_3

In [14]:
# fig, ax = plt.subplots(nrows=1, ncols=3)
# ax1, ax2, ax3 = ax
# fig.subplots_adjust(wspace=0.3)
# fig.set_size_inches(20,10)

# ax1.hist(train_df["Ecology_1"], bins=20, density=True)
# ax1.set_ylabel("Density", fontdict={"fontsize": 15})
# ax1.set_xlabel("Ecology_1", fontdict={"fontsize": 15})
# ax1.set_title("Ecology 1 distribushion", fontdict={"fontsize": 15})

# ax2.scatter(train_df["DistrictId"], train_df["Ecology_1"], s=2.5)
# ax2.set_ylabel("Ecology_1", fontdict={"fontsize": 15})
# ax2.set_xlabel("District Id", fontdict={"fontsize": 15})
# ax2.set_title("Ecology_1 vs District_ID", fontdict={"fontsize": 15})

# ax3.scatter(train_df["Ecology_1"], train_df["Price"], s=2.1)
# ax3.set_title("Ecology_1 vs Price", fontdict={"fontsize": 15})
# ax3.set_xlabel("Ecology_1", fontdict={"fontsize": 15})
# ax3.set_ylabel("Price", fontdict={"fontsize": 15})

# fig.savefig("graphs/4._Graph_4.jpg")

Исходя из графиков можно сделать вывод о том что уровень показателя Ecology_1 не зависит от конкретного района. Собственно как и показатель цены не имеет какой-то ярко выраженной зависимости. Проанализируем категориальные показатели Ecology_2 и Ecology_3

In [15]:
# fig, ax = plt.subplots(nrows=1, ncols=2)
# ax1, ax2 = ax
# fig.set_size_inches(20, 10)
# fig.subplots_adjust(wspace=0.3)

# ax1.scatter(train_df.loc[train_df["Ecology_2"] == "B", "DistrictId"],
#             train_df.loc[train_df["Ecology_2"] == "B", "Price"],
#             s=2.5, c="y", label="B")
# ax1.scatter(train_df.loc[train_df["Ecology_2"] == "A", "DistrictId"],
#             train_df.loc[train_df["Ecology_2"] == "A", "Price"],
#             s=2.5, c="red", label="A")
# ax1.legend(fontsize=20, markerscale=9)
# ax1.set_xlabel("District ID", fontdict={"fontsize": 15})
# ax1.set_ylabel("Price", fontdict={"fontsize": 15})
# ax1.set_title("Price vs District_Id with Ecology_2", fontdict={"fontsize": 15})

# ax2.scatter(train_df.loc[train_df["Ecology_3"] == "B", "DistrictId"],
#             train_df.loc[train_df["Ecology_3"] == "B", "Price"],
#             s=2.5, c="y", label="B")
# ax2.scatter(train_df.loc[train_df["Ecology_3"] == "A", "DistrictId"],
#             train_df.loc[train_df["Ecology_3"] == "A", "Price"],
#             s=2.5, c="red", label="A")
# ax2.legend(fontsize=20, markerscale=9)
# ax2.set_title("Price vs District_Id with Ecology_3", fontdict={"fontsize": 15})
# ax2.set_xlabel("District ID", fontdict={"fontsize": 15})
# ax2.set_ylabel("Price", fontdict={"fontsize": 15})

# fig.savefig("graphs/5._Graph_5.jpg")

Как видно из графиков переменные не сбалансированы. Граддация переменной Ecology_2 по категории А в основном распределена в одном конкретном районе. Ecology_3 так же не вносит никакой ясности в распределение цен или районов. Исходя из этого предлагаю убрать данные переменные из последующего анализа и прогнозирования.

Проанализируем переменные Social_1, Social_2, Social_3

In [16]:
# fig, ax = plt.subplots(nrows=2, ncols=2)
# ax1, ax2, ax3, ax4 = ax.flatten()
# fig.set_size_inches(30,15)
# fig.subplots_adjust(hspace=0.3, wspace=0.3)

# ax1.hist(train_df["Social_1"].to_numpy(), bins=100, density=True)
# ax1.set_title("Social_1", fontdict={"fontsize": 20})

# ax2.scatter(train_df["Social_1"], train_df["Price"], s=2.5)
# ax2.set_title("Social_1 vs Price", fontdict={"fontsize": 20})

# ax3.hist(train_df.loc[train_df["DistrictId"] <=50, "Social_1"].to_numpy(), 
#          bins=100, density=True, color="r", label="District <= 50")
# ax3.hist(train_df.loc[train_df["DistrictId"] >50, "Social_1"].to_numpy(), 
#          bins=100, density=True, color="g", label="District > 50", alpha=0.7)
# ax3.legend(fontsize=20, markerscale=9)
# ax3.set_title("Social_1 vs District_Id", fontdict={"fontsize": 20})

# ax4.scatter(train_df.loc[train_df["DistrictId"] <=50, "Social_1"].to_numpy(),
#             train_df.loc[train_df["DistrictId"] <=50, "Price"].to_numpy(), s=2.5, c="r", label="District <= 50")
# ax4.scatter(train_df.loc[train_df["DistrictId"] >50, "Social_1"].to_numpy(),
#             train_df.loc[train_df["DistrictId"] >50, "Price"].to_numpy(), s=2.5, c="g", label="District > 50")
# ax4.legend(markerscale=9, fontsize=20)
# ax4.set_title("Social_1 vs Price and District", fontdict={"fontsize": 20})

# fig.savefig("graphs/6._Graph_6.jpg")

In [17]:
# fig, ax = plt.subplots(nrows=2, ncols=2)
# ax1, ax2, ax3, ax4 = ax.flatten()
# fig.set_size_inches(20,10)
# fig.subplots_adjust(hspace=0.3, wspace=0.3)

# ax1.hist(train_df["Social_2"].to_numpy(), bins=100, density=True)
# ax1.set_title("Social_2", fontdict={"fontsize": 15})

# ax2.scatter(train_df["Social_2"], train_df["Price"], s=2.5)
# ax2.set_title("Social_2 vs Price", fontdict={"fontsize": 15})

# ax3.hist(train_df.loc[train_df["DistrictId"] <=50, "Social_2"].to_numpy(), 
#          bins=100, density=True, color="r", label="District <= 50")
# ax3.hist(train_df.loc[train_df["DistrictId"] >50, "Social_2"].to_numpy(), 
#          bins=100, density=True, color="g", label="District > 50", alpha=0.7)
# ax3.legend()
# ax3.set_title("Social_2 vs District_Id", fontdict={"fontsize": 15})

# ax4.scatter(train_df.loc[train_df["DistrictId"] <=50, "Social_2"].to_numpy(),
#             train_df.loc[train_df["DistrictId"] <=50, "Price"].to_numpy(), s=2.5, c="r", label="District <= 50")
# ax4.scatter(train_df.loc[train_df["DistrictId"] >50, "Social_2"].to_numpy(),
#             train_df.loc[train_df["DistrictId"] >50, "Price"].to_numpy(), s=2.5, c="g", label="District > 50")
# ax4.legend(markerscale=9)
# ax4.set_title("Social_2 vs Price and District", fontdict={"fontsize": 15})

# fig.savefig("graphs/7._Graph_7.jpg")

In [18]:
# fig, ax = plt.subplots(nrows=2, ncols=2)
# ax1, ax2, ax3, ax4 = ax.flatten()
# fig.set_size_inches(20,10)
# fig.subplots_adjust(hspace=0.3, wspace=0.3)

# ax1.hist(train_df["Social_3"].to_numpy(), bins=100, density=True)
# ax1.set_title("Social_3", fontdict={"fontsize": 15})

# ax2.scatter(train_df["Social_3"], train_df["Price"], s=2.5)
# ax2.set_title("Social_3 vs Price", fontdict={"fontsize": 15})

# ax3.hist(train_df.loc[train_df["DistrictId"] <=50, "Social_3"].to_numpy(), 
#          bins=100, density=True, color="r", label="District <= 50")
# ax3.hist(train_df.loc[train_df["DistrictId"] >50, "Social_3"].to_numpy(), 
#          bins=100, density=True, color="g", label="District > 50", alpha=0.7)
# ax3.legend()
# ax3.set_title("Social_3 vs District_Id", fontdict={"fontsize": 15})

# ax4.scatter(train_df.loc[train_df["DistrictId"] <=50, "Social_3"].to_numpy(),
#             train_df.loc[train_df["DistrictId"] <=50, "Price"].to_numpy(), s=2.5, c="r", label="District <= 50")
# ax4.scatter(train_df.loc[train_df["DistrictId"] >50, "Social_3"].to_numpy(),
#             train_df.loc[train_df["DistrictId"] >50, "Price"].to_numpy(), s=2.5, c="g", label="District > 50")
# ax4.legend(markerscale=9)
# ax4.set_title("Social_3 vs Price and District", fontdict={"fontsize": 15})

# fig.savefig("graphs/8._Graph_8.jpg")

Представленные графики не вносят определенной ясности в распределение показателей social по районам. Чтобы окончательно убедится в отсутсвии/наличии взаимосвязей, построим трехмерную диаграмму с осями social_1, social_2 и social_3. Но для начала приведем данные к стандартной форме с средним 0 и ско 1, за исключением данных по шкале Social_3 - она имеет ранговую градацию

In [19]:
from sklearn.preprocessing import StandardScaler

In [20]:
stscaler = StandardScaler()

In [21]:
Social_1_d1 = train_df.loc[train_df["DistrictId"] <=50, "Social_1"].to_numpy().reshape(-1, 1)
Social_2_d1 = train_df.loc[train_df["DistrictId"] <=50, "Social_2"].to_numpy().reshape(-1, 1)
Social_3_d1 = train_df.loc[train_df["DistrictId"] <=50, "Social_3"].to_numpy().reshape(-1, 1)

Social_1_d1 = stscaler.fit_transform(Social_1_d1)
Social_2_d1 = stscaler.fit_transform(Social_2_d1)

In [22]:
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# fig.set_size_inches(20,10)

# ax.scatter(Social_1_d1, Social_2_d1, Social_3_d1, c="r", marker="v") 
# ax.set_xlabel("Social_1", fontdict={"fontsize": 15})
# ax.set_ylabel("Social_2", fontdict={"fontsize": 15})
# ax.set_zlabel("Social_3", fontdict={"fontsize": 15})

# fig.savefig("graphs/9._Graph_9.jpg")

Как видно из графика покзаатели Social_1 и Social_2 имеют ярковыраженную коррекляцию. Ниже посчитаем попарные коэффициенты корреляции.

In [23]:
np.corrcoef(np.array([Social_1_d1.flatten(), Social_2_d1.flatten(), Social_3_d1.flatten()]))

array([[1.        , 0.97406192, 0.06244278],
       [0.97406192, 1.        , 0.11675252],
       [0.06244278, 0.11675252, 1.        ]])

Как видно показатели Social_1 и Social_2 имеют максимальную корреляцию. Имеет смысл выкинуть какой либбо из показателей из дальнейшего анализа. Посмотрим на описательные статистики для Social_1 и Social_2

In [24]:
pd.DataFrame({"Social_1": Social_1_d1.reshape(-1, 1).flatten(), 
              "Social_2": Social_2_d1.reshape(-1, 1).flatten()}).describe().round(7)

,Social_1,Social_2
count,"5,876.000","5,876.000"
mean,0.000,0.000
std,1.000,1.000
min,-1.124,-1.039
25%,-0.852,-0.798
50%,0.072,-0.069
75%,0.615,0.376
max,2.897,3.256


Особой разницы в распределении двух переменных я не вижу. По крайней мере если оценивать по квантилям распределение данных похожее. Для дальнейшего анализа предлагаю исключить показатель Social_2. Таким образом для кластера с районами =< 50 убираем показатель Social_2. Сделаем такой же анализ для кластера с районами > 50

In [25]:
Social_1_d2 = train_df.loc[train_df["DistrictId"] >50, "Social_1"].to_numpy().reshape(-1, 1)
Social_2_d2 = train_df.loc[train_df["DistrictId"] >50, "Social_2"].to_numpy().reshape(-1, 1)
Social_3_d2 = train_df.loc[train_df["DistrictId"] >50, "Social_3"].to_numpy().reshape(-1, 1)

Social_1_d2 = stscaler.fit_transform(Social_1_d2)
Social_2_d2 = stscaler.fit_transform(Social_2_d2)

In [26]:
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# fig.set_size_inches(20,10)

# ax.scatter(Social_1_d2, Social_2_d2, Social_3_d2, c="r", marker="v") 
# ax.set_xlabel("Social_1", fontdict={"fontsize": 15})
# ax.set_ylabel("Social_2", fontdict={"fontsize": 15})
# ax.set_zlabel("Social_3", fontdict={"fontsize": 15})

# fig.savefig("graphs/10._Graph_10.jpg")

In [27]:
np.corrcoef(np.array([Social_1_d1.flatten(), Social_2_d1.flatten(), Social_3_d1.flatten()]))

array([[1.        , 0.97406192, 0.06244278],
       [0.97406192, 1.        , 0.11675252],
       [0.06244278, 0.11675252, 1.        ]])

Исходя из графика и корреляционной матрицы, для данной группы наблюдений можно так же спокойно исключить показатель Social_2. Показатель Social_3 для обоих кластеров имеет миниальный уровень разброса. Поскольку он имеет градацию, проанализируем ее для каждого кластера

In [28]:
train_df.loc[:, "Social_3"].value_counts()

0      2997
1      1861
3      1136
4       835
2       635
6       423
5       391
141     245
9       192
7       184
11      172
8       170
10      142
16      101
14       97
27       59
48       58
56       49
84       43
24       37
87       27
59       25
93       25
19       22
20       19
23       18
39       15
73       10
45        9
37        3
Name: Social_3, dtype: int64

Как видно показатель Social_3 имеет наибольшее количество наблюдений по градациям от 1 до 6. Поэтому предлагаю сделать следующее преоразованию. Градацию Social_3 разбить на бинарный показатель. Признаки от 1 до 6 будут входить в группу 0, а остальные в группу 1. Проанализируем признаки Healthcare

In [29]:
train_df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.000,47.982,29.443,6.000,7,9.000,1969,0.089,B,B,33,7976,5,nan,0,11,B,"184,966.931"
1,15053,41,3.000,65.684,40.050,8.000,7,9.000,1978,0.000,B,B,46,10309,1,240.000,1,16,B,"300,009.450"
2,4765,53,2.000,44.948,29.198,0.000,8,12.000,1968,0.050,B,B,34,7759,0,229.000,1,3,B,"220,925.909"
3,5809,58,2.000,53.353,52.732,9.000,8,17.000,1977,0.438,B,B,23,5735,3,"1,084.000",0,5,B,"175,616.227"
4,10783,99,1.000,39.649,23.776,7.000,11,12.000,1976,0.012,B,B,35,5776,1,"2,078.000",2,4,B,"150,226.532"


In [30]:
train_df["Healthcare_1"].value_counts()

540.000      511
30.000       348
1,046.000    245
750.000      163
229.000      148
            ... 
370.000       14
32.000        12
1,815.000     10
35.000         2
0.000          1
Name: Healthcare_1, Length: 79, dtype: int64

In [31]:
train_df["Helthcare_2"].value_counts()

0    4214
1    2183
3    1670
2    1158
5     388
4     288
6      99
Name: Helthcare_2, dtype: int64

In [32]:
health_1_1 = train_df.loc[train_df["DistrictId"] <= 50, "Healthcare_1"].to_numpy()
health_1_2 = train_df.loc[train_df["DistrictId"] > 50, "Healthcare_1"].to_numpy()

In [33]:
# fig, ax = plt.subplots(nrows=1, ncols=1)
# fig.set_size_inches(20,10)

# ax.hist(health_1_2, bins=100, density=True, color="b", label="Healthcare_1 > 50")
# ax.hist(health_1_1, bins=100, density=True, color="g", label="Healthcare_1 <= 50", alpha=0.7)
# ax.legend(fontsize=10, markerscale=9)
# ax.set_title("Distribushion of price vs Healthcare_1", fontdict={"fontsize": 15})
# ax.set_xlabel("Healthcare_1", fontdict={"fontsize": 15})
# ax.set_ylabel("Dencity", fontdict={"fontsize": 15})


# fig.savefig("graphs/11._Graph_11.jpg")

In [34]:
pd.DataFrame({"District <= 50": health_1_1}).describe()

,District <= 50
count,"2,522.000"
mean,878.587
std,746.866
min,0.000
25%,320.000
50%,717.500
75%,"1,130.000"
max,"4,087.000"


In [35]:
pd.DataFrame({"District > 50": health_1_2}).describe()

,District > 50
count,"2,680.000"
mean,"1,391.639"
std,"1,171.866"
min,30.000
25%,350.000
50%,"1,084.000"
75%,"1,970.000"
max,"4,849.000"


Какой-то взаимосвязи между районами и показателем Healthcare_1 обнаружить не удалось. Попробуем сделать следующий трюк. Скорее всего показатели Heakthcare_1 и Helthcare_2 связаны между собой. Можно построить гистаграммы частот распределения показателя Helthcare_1 в зависимости от градации Helathcare_2. И далее можно заменить Nan значения средним или медианой в зависимости от показателя Healthcare_2. При этом, если данное предположение окажется верно, можно будет убрать показатель Healthcare_2 из дальнейшего анализа.

In [36]:
health_df = train_df.loc[train_df["Healthcare_1"].notnull(), ["Healthcare_1", "Helthcare_2"]]

In [37]:
health_df.head()

,Healthcare_1,Helthcare_2
1,240.000,1
2,229.000,1
3,"1,084.000",0
4,"2,078.000",2
5,990.000,0


In [38]:
health_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5202 entries, 1 to 9999
Data columns (total 2 columns):
Healthcare_1    5202 non-null float64
Helthcare_2     5202 non-null int64
dtypes: float64(1), int64(1)
memory usage: 121.9 KB


In [39]:
# fig, ax = plt.subplots(nrows=4, ncols=2)
# fig.set_size_inches(20,10)
# fig.subplots_adjust(wspace=0.3, hspace=0.3)
# ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8 = ax.flatten()

# ax1.hist(health_df.loc[health_df["Helthcare_2"] == 0, "Healthcare_1"], bins=100, density=True)
# ax1.set_title("Healthcare_2 = 0", fontdict={"fontsize": 15})

# ax2.hist(health_df.loc[health_df["Helthcare_2"] == 1, "Healthcare_1"], bins=100, density=True)
# ax2.set_title("Healthcare_2 = 1", fontdict={"fontsize": 15})

# ax3.hist(health_df.loc[health_df["Helthcare_2"] == 2, "Healthcare_1"], bins=100, density=True)
# ax3.set_title("Healthcare_2 = 2", fontdict={"fontsize": 15})

# ax4.hist(health_df.loc[health_df["Helthcare_2"] == 3, "Healthcare_1"], bins=100, density=True)
# ax4.set_title("Healthcare_2 = 3", fontdict={"fontsize": 15})

# ax5.hist(health_df.loc[health_df["Helthcare_2"] == 4, "Healthcare_1"], bins=100, density=True)
# ax5.set_title("Healthcare_2 = 4", fontdict={"fontsize": 15})

# ax6.hist(health_df.loc[health_df["Helthcare_2"] == 5, "Healthcare_1"], bins=100, density=True)
# ax6.set_title("Healthcare_2 = 5", fontdict={"fontsize": 15})

# ax7.hist(health_df.loc[health_df["Helthcare_2"] == 6, "Healthcare_1"], bins=100, density=True)
# ax7.set_title("Healthcare_2 = 6", fontdict={"fontsize": 15})

# fig.savefig("graphs/12._Graph_12.jpg")

Как видно чем выше класс Healthcare_2 тем более разряженные данные попадаются. Распределение неравномерно и имеет большое количество пиков при этом область определения распределения имеет очень большой размах. На мой взггляд в данном случае логично было бы использовать не среднее с медианой, а моду, поскольку для каждого класса встречается хотя бы один существенный пик Проанализируем еще раз Helathcare_2 в разрезе пропущенных значений

In [40]:
print("Заполненные значения")
health_df["Helthcare_2"].value_counts()

Заполненные значения


3    1347
0    1275
2    1056
1     925
4     288
5     212
6      99
Name: Helthcare_2, dtype: int64

In [41]:
print("Пропущенные значения")
train_df.loc[train_df["Healthcare_1"].isnull(), "Helthcare_2"].value_counts()

Пропущенные значения


0    2939
1    1258
3     323
5     176
2     102
Name: Helthcare_2, dtype: int64

Как видно в основном пропущенные значения встречаются по тем классам, где имеется достаточно большое количество заполненных данных. 0, 1, 3. Так же если просмотреть гистограммы на графике выше каждый класс имеет свой уровень моды который его характеризует. Выведем моды для каждого класса

In [42]:
list_of_mode = []
for i in range(7):
    tmp_mode = health_df.loc[health_df["Helthcare_2"] == i, "Healthcare_1"].mode()[0]
    list_of_mode.append(tmp_mode)
    
for i in range(7):
    print(f"Mode for healthcare_2 = {i}: {list_of_mode[i]}")

Mode for healthcare_2 = 0: 540.0
Mode for healthcare_2 = 1: 229.0
Mode for healthcare_2 = 2: 30.0
Mode for healthcare_2 = 3: 1046.0
Mode for healthcare_2 = 4: 1894.0
Mode for healthcare_2 = 5: 176.0
Mode for healthcare_2 = 6: 645.0


Для дальнейшего анализа полученные моды проставим в пустые значения по показателю Healthcar_1. Проанализируем последние показатели районов Shops_1 и Shops_2

In [43]:
train_df["Shops_1"].value_counts()

0     2316
1     1325
5     1324
2      964
4      841
3      711
6      487
11     401
9      316
7      294
8      253
23     245
10     215
15     176
16      83
19      49
Name: Shops_1, dtype: int64

In [44]:
train_df["Shops_2"].value_counts()

B    9175
A     825
Name: Shops_2, dtype: int64

Как видно последний покзаатель существенно несбалансирован, поэтому предлагаю его убрать из последующего анализа. Показатель Shops_1 в основном распределен в градаиях от 1 до 6. Предлагаю его так же превратить в бинарный признак. Где 0 будет у наблюдений от 1 до 6, а 1 - у всех остальных. Для дальнейших преобразований напишем функцию, принимающую на вход фрейм и выводящую обработанные данные по районам.

In [45]:
train_df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.000,47.982,29.443,6.000,7,9.000,1969,0.089,B,B,33,7976,5,nan,0,11,B,"184,966.931"
1,15053,41,3.000,65.684,40.050,8.000,7,9.000,1978,0.000,B,B,46,10309,1,240.000,1,16,B,"300,009.450"
2,4765,53,2.000,44.948,29.198,0.000,8,12.000,1968,0.050,B,B,34,7759,0,229.000,1,3,B,"220,925.909"
3,5809,58,2.000,53.353,52.732,9.000,8,17.000,1977,0.438,B,B,23,5735,3,"1,084.000",0,5,B,"175,616.227"
4,10783,99,1.000,39.649,23.776,7.000,11,12.000,1976,0.012,B,B,35,5776,1,"2,078.000",2,4,B,"150,226.532"


Создадим функцию по обработке фрейма, которая будет принимать на вход два фрейма. Один - тот который нужно обработать. Второй - тот из которого нужно вытянуть усредненные данные

In [46]:
def preliminary_preprocessing(df, df_for_calc):
    output_df = df.copy()
    
    # поменяем градацию по District ID
    output_df.loc[output_df["DistrictId"] <= 50, "DistrictId"] = 0
    output_df.loc[output_df["DistrictId"] > 50, "DistrictId"] = 1
    
    # Убираем Ecology_2, Ecology_3, Social_2, Shops_2
    output_df.drop(columns=["Ecology_2", "Ecology_3", "Social_2", "Shops_2"], inplace=True)
    
    # Преобразуем Social_3 показатели от 0 до 6 вклюительно - градация 0, остальные - 1
    output_df.loc[output_df["Social_3"] <= 6, "Social_3"] = 0
    output_df.loc[output_df["Social_3"] > 6, "Social_3"] = 1
    
    #Вставим пропущенные моды на позиции Healthcare_1
    list_of_mode = []
    for i in range(7):
        tmp_mode = df_for_calc.loc[df_for_calc["Helthcare_2"] == i, "Healthcare_1"].mode()[0]
        list_of_mode.append(tmp_mode)
    
    for i in range(7):
        output_df.loc[(output_df["Helthcare_2"] == i) & (output_df["Healthcare_1"].isnull()), 
                      "Healthcare_1"] = list_of_mode[i]
    
    #Изменим градацию Shops_1
    output_df.loc[output_df["Shops_1"] <= 6, "Shops_1"] = 0
    output_df.loc[output_df["Shops_1"] > 6, "Shops_1"] = 1
    
        
    # Так же немного изменим показатель HouseYear. Сделаем не дату ввода дома в эксплуатацию, а возраст. Взяв за основу текущий год
    output_df["HouseYear"] = 2020 - output_df["HouseYear"]
    
    
    #Заменим показатель Floor на две других фичи. Первая - евляется ли этаж лота первым по отношению к дому Где 0 и 1 соответсвенно нет или да
    #Вторая - является ли этаж лота последним в доме, где 0 и 1 соответсвенно да или нет
    output_df["Is_first_floor"] = 0
    output_df.loc[output_df["Floor"] == 1, "Is_first_floor"] = 1
    
    output_df["Is_last_floor"] = 0
    output_df.loc[output_df["HouseFloor"] == output_df["Floor"], "Is_last_floor"] = 1
    
    output_df.drop(columns=["Floor"], inplace=True)
    
    output_df = output_df[["DistrictId", "Rooms", "Square", "LifeSquare", "KitchenSquare", "HouseYear", "Is_first_floor", 
                           "Is_last_floor", "HouseFloor", "Ecology_1", "Social_1", "Social_3", "Helthcare_2", 
                           "Shops_1"]]
    
    
    return output_df


In [47]:
preprocessing_df = preliminary_preprocessing(train_df, train_df)

In [48]:
preprocessing_df.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear,Is_first_floor,Is_last_floor,HouseFloor,Ecology_1,Social_1,Social_3,Helthcare_2,Shops_1
0,0,2.000,47.982,29.443,6.000,51,0,0,9.000,0.089,33,0,0,1
1,0,3.000,65.684,40.050,8.000,42,0,0,9.000,0.000,46,0,1,1
2,1,2.000,44.948,29.198,0.000,52,0,0,12.000,0.050,34,0,1,0
3,1,2.000,53.353,52.732,9.000,43,0,0,17.000,0.438,23,0,0,0
4,1,1.000,39.649,23.776,7.000,44,0,0,12.000,0.012,35,0,2,0


Проанализируем остальные признаки относящиеся непосредственно к лотам

In [49]:
preprocessing_df.loc[:, "DistrictId":"HouseYear"].describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear
count,"10,000.000","10,000.000","10,000.000","7,887.000","10,000.000","10,000.000"
mean,0.412,1.891,56.316,37.200,6.273,"-1,970.166"
std,0.492,0.840,21.059,86.241,28.561,"200,500.261"
min,0.000,0.000,1.137,0.371,0.000,"-20,049,991.000"
25%,0.000,1.000,41.775,22.770,1.000,19.000
50%,0.000,2.000,52.513,32.781,6.000,43.000
75%,1.000,2.000,65.901,45.129,9.000,46.000
max,1.000,19.000,641.065,"7,480.592","2,014.000",110.000


По показзателю Rooms есть данные где минимальный уровень количества комнат равен 0. количество в 19 комнат тоже странное, но наверное такие лоты есть....верхние выбросы не будем трогать.   
По покзателю Square тоже странные нижние границы, верхние в принципе не вызывают вопросов.  
По показателю LifeSquare странные границы и в верху и внизу.  
По показателю KitchenSquare странные границы и в верху и внизу Floor - нормально  
HouseFloor - пусть останется так  
HouseYear - явная ошибка в минимальной границе.  


In [50]:
preprocessing_df["Rooms"].value_counts()

2.000     3880
1.000     3705
3.000     2235
4.000      150
5.000       18
0.000        8
10.000       2
19.000       1
6.000        1
Name: Rooms, dtype: int64

Смущают показатели по Rooms, где количество комнат равно 10, 19, 6 и 0. Провалимся туда, посмотрим

In [51]:
preprocessing_df[preprocessing_df["Rooms"] >= 6]

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear,Is_first_floor,Is_last_floor,HouseFloor,Ecology_1,Social_1,Social_3,Helthcare_2,Shops_1
377,1,10.000,59.057,36.223,10.000,18,0,1,22.000,0.091,74,0,5,1
1454,0,19.000,42.006,21.779,7.000,6,0,1,17.000,0.007,1,0,0,0
2170,1,6.000,59.414,38.702,6.000,51,0,0,9.000,0.033,66,0,3,1
8849,0,10.000,60.871,38.421,10.000,26,0,0,2.000,0.162,25,0,2,0


Заменим на моду от греха подальше. Для преобразования датафрейма создадим функцию

Построим гисторграмму по фиче Square

In [52]:
# fig, ax = plt.subplots()
# fig.set_size_inches(20,10)

# ax.hist(preprocessing_df["Square"].to_numpy(), bins=1000, density=True)
# ax.set_title("Square hist", fontdict={"fontsize": 15})
# ax.set_xlim(0, 200)

# fig.savefig("graphs/13._Graph_13.jpg")

Из графика видно, что есть наблюдения меньше 25 кв.м. и больше 200. Оценим сколько таких наблюдений

In [53]:
preprocessing_df[preprocessing_df["Square"] < 25].shape[0],\
preprocessing_df[preprocessing_df["Square"] > 200].shape[0]

(34, 6)

Заменим в них данные соответсвенно на 25 и 200 с помощью функции

Тем же способом проанализируем LifeSquare

In [54]:
# fig, ax = plt.subplots()
# fig.set_size_inches(20,10)

# ax.hist(preprocessing_df.loc[(preprocessing_df["LifeSquare"] >= 10) & 
#                              (preprocessing_df["LifeSquare"] <= 100), "LifeSquare"].to_numpy(), bins=100, density=True)
# ax.set_title("LifeSquare hist", fontdict={"fontsize": 15})

# fig.savefig("graphs/14._Graph_14.jpg")

Заменим в них данные соответсвенно на 10 и 100 с помощью функции

Проанализируем KitchenSquare

In [55]:
preprocessing_df["KitchenSquare"].value_counts()

1.000        2460
8.000        1306
5.000        1169
10.000       1075
6.000        1038
9.000         843
0.000         697
7.000         609
12.000        249
11.000        233
13.000         67
14.000         51
4.000          39
15.000         31
3.000          22
16.000         16
20.000         14
17.000         12
19.000         11
18.000          6
2.000           4
22.000          3
41.000          2
112.000         2
25.000          2
43.000          2
51.000          2
37.000          2
32.000          2
30.000          2
58.000          2
72.000          1
96.000          1
66.000          1
48.000          1
40.000          1
2,014.000       1
35.000          1
60.000          1
78.000          1
27.000          1
84.000          1
62.000          1
42.000          1
63.000          1
39.000          1
1,970.000       1
36.000          1
75.000          1
26.000          1
21.000          1
29.000          1
23.000          1
73.000          1
123.000         1
31.000    

Уберем все наблюдения, больше 20 и заменим на моду. Поскольку большинство наблюдений находятся в пределах от 1 до 20 и большинство наблюдений составляет 1, а так же тот факт, что все наблюдения целые числа говорит о том, что скорее всего этот признак является ранговым и отражает не площадь кухни, а нечто другое лиибо отношение каких-то площадей, либо еще что-то

Осталось оценить HouseYear. Посмотрим значения ниже нуля

In [56]:
preprocessing_df[preprocessing_df["HouseYear"] < 0]

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear,Is_first_floor,Is_last_floor,HouseFloor,Ecology_1,Social_1,Social_3,Helthcare_2,Shops_1
1497,1,1.000,37.265,20.240,9.000,-20049991,0,0,12.000,0.136,30,1,3,0
4189,1,2.000,44.792,28.360,5.000,-2948,0,0,9.000,0.320,25,1,5,1


Заменим на моду через функцию

Просмотрим 10 самых старых наблюдений

In [57]:
preprocessing_df.loc[preprocessing_df["HouseYear"].sort_values()[-10:].index, :]

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear,Is_first_floor,Is_last_floor,HouseFloor,Ecology_1,Social_1,Social_3,Helthcare_2,Shops_1
5016,1,2.000,67.839,65.200,9.000,102,0,0,8.000,0.015,17,1,0,1
1386,1,2.000,76.083,48.026,14.000,102,0,0,5.000,0.037,32,1,4,1
8144,1,2.000,54.751,31.515,10.000,103,0,0,4.000,0.000,22,1,3,1
5053,1,1.000,35.508,20.313,4.000,103,0,0,5.000,0.000,22,1,3,1
1380,1,3.000,75.565,44.873,9.000,103,0,0,7.000,0.015,17,1,0,1
7611,1,3.000,102.964,48.322,14.000,104,0,0,6.000,0.037,32,1,4,1
910,1,2.000,56.172,28.384,7.000,106,0,0,6.000,0.000,7,1,1,0
5111,1,1.000,63.293,29.390,18.000,108,0,0,9.000,0.014,19,0,0,0
2656,1,4.000,151.151,98.187,23.000,108,0,0,7.000,0.037,32,1,4,1
9317,1,1.000,50.809,15.920,20.000,110,0,0,9.000,0.014,19,0,0,0


Ну в целом относительно понятно. Заменять ничего не будем. Только теперь увидел еще один недостаток данных. LifeSquare может быть больше Square. Исправим это. Вычислим среднюю разницу между Square и LifeSquare для каждой градации комнат и используем этот показатель для замены данных

Так же у нас имеется достаточное количество пропущенных наблюдений по фиче LifeSquare. Забегая вперед скажу, что я пытался обучить модель на части параметров, но ни линейная, ни ближайщий сосед, ни случайный лес не дали достаточного уровня R2 (получалось ниже 70%). Поэтому сделаем следующее Для нулевых значений мы знаем какое количество комнат, посчитаем среднее для наблюдений где данные эти есть и вставим их вместо нулевых.

Создадим вторую функцию по обработке фрейма

In [58]:
def preliminary_preprocessing_2(df, df_for_calc):
    output_df = df.copy()
    
    #Заменяем значения по фиче rooms на моду 
    output_df.loc[output_df["Rooms"] >= 6, "Rooms"] = df_for_calc["Rooms"].mode()[0]
    output_df.loc[output_df["Rooms"] == 0, "Rooms"] = df_for_calc["Rooms"].mode()[0]
    
    #Заменяем выбросы по фиче Square
    output_df.loc[output_df["Square"] < 25, "Square"] = 25
    output_df.loc[output_df["Square"] > 200, "Square"] = 200
    
    #Заменим выбросы по фиче LifeSquare
    output_df.loc[output_df["LifeSquare"] <= 10, "LifeSquare"] = 10
    output_df.loc[output_df["LifeSquare"] >= 100, "LifeSquare"] = 100
    
    #Заменим выбросы по фиче KitchenSquare на моду
    output_df.loc[output_df["KitchenSquare"] > 20, "KitchenSquare"] = \
        df_for_calc.loc[df_for_calc["KitchenSquare"] <= 20, "KitchenSquare"].mode()[0]
    
    #Заменим отрицательные даты по фиче HouseYear на моду
    output_df.loc[output_df["HouseYear"] < 0, "HouseYear"] =\
        (2020 - df_for_calc.loc[(df_for_calc["HouseYear"] >= 0) & 
                                ((2020 - df_for_calc["HouseYear"]) <= 110), "HouseYear"].mode()[0])
    #Так же заменим на всякий случай значения, показатели которых больше 110 - максимального возраста дома в доступных наблюдениях
    output_df.loc[output_df["HouseYear"] > 110, "HouseYear"] =\
        (2020 - df_for_calc.loc[(df_for_calc["HouseYear"] >= 0) & 
                                ((2020 - df_for_calc["HouseYear"]) <= 110), "HouseYear"].mode()[0])
        
    #Исправим разницу между Square и LifeSquare
    diff_in_square = {}
    for room in df_for_calc["Rooms"].value_counts().index:
        diff = (df_for_calc.loc[df_for_calc["Rooms"] == room, "Square"] -\
                df_for_calc.loc[df_for_calc["Rooms"] == room, "LifeSquare"]).mean()
        diff_in_square[room] = diff
        
    for room in output_df["Rooms"].value_counts().index:
        room_filter = [0, ]
        while len(room_filter) != 0:
            room_filter = output_df.loc[(output_df["LifeSquare"] > output_df["Square"]) & 
                                        (output_df["Rooms"] == room), :].index 
            output_df.loc[room_filter, "LifeSquare"] = output_df.loc[room_filter, "LifeSquare"] - diff_in_square[room]
            
    #Заменим пропущенные значения по фиче LifeSquare
    for room in list(output_df["Rooms"].value_counts().index):
        output_df.loc[(output_df["LifeSquare"].isnull()) & (output_df["Rooms"] == room), "LifeSquare"] =\
            df_for_calc.loc[(df_for_calc["LifeSquare"].notnull()) & (df_for_calc["Rooms"] == room), "LifeSquare"].mean()
        
    return output_df

In [59]:
preprocessing_df = preliminary_preprocessing_2(preprocessing_df, train_df)

In [60]:
preprocessing_df.loc[:, "DistrictId":"HouseYear"].describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear
count,"10,000.000","10,000.000","10,000.000","10,000.000","10,000.000","10,000.000"
mean,0.412,1.888,56.237,35.097,5.652,35.136
std,0.492,0.811,19.156,14.845,3.888,18.410
min,0.000,1.000,25.000,10.000,0.000,0.000
25%,0.000,1.000,41.775,24.914,1.000,19.000
50%,0.000,2.000,52.513,32.804,6.000,43.000
75%,1.000,2.000,65.901,43.396,9.000,46.000
max,1.000,5.000,200.000,100.000,20.000,110.000


In [61]:
preprocessing_df.head(5)

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear,Is_first_floor,Is_last_floor,HouseFloor,Ecology_1,Social_1,Social_3,Helthcare_2,Shops_1
0,0,2.000,47.982,29.443,6.000,51,0,0,9.000,0.089,33,0,0,1
1,0,3.000,65.684,40.050,8.000,42,0,0,9.000,0.000,46,0,1,1
2,1,2.000,44.948,29.198,0.000,52,0,0,12.000,0.050,34,0,1,0
3,1,2.000,53.353,52.732,9.000,43,0,0,17.000,0.438,23,0,0,0
4,1,1.000,39.649,23.776,7.000,44,0,0,12.000,0.012,35,0,2,0


В нашем датафрейме содержаться следующие категориальные переменные:  
1. DistrictID у которого значения <= 50 равны 0, > 50 = 1  
2. Social_3 значения <= 6 равны 0, > 6 = 1  
3. Shops_1 значения <= 6 равны 0, > 6 = 1  
4. Is_first_floor - 1 если этаж первый, 0 - если нет  
5. Is_last_floor - 1 если этаж последний, 0 - если нет  
Последний обзор фрейма перед построением модели

### Строим модель

Разобьем изначальный train_df на обучающий и тестовый

In [62]:
from sklearn.model_selection import train_test_split

In [63]:
X_train, X_test, y_train, y_test = train_test_split(train_df.loc[:, "DistrictId":"Shops_2"],
                                                    train_df["Price"], test_size=0.3, random_state=42)

In [64]:
(X_train.shape, y_train.shape),\
(X_test.shape, y_test.shape)

(((7000, 18), (7000,)), ((3000, 18), (3000,)))

In [65]:
X_train.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
9069,23,1.000,41.681,22.796,8.000,14,17.000,2015,0.076,B,B,6,1437,3,nan,0,2,B
2603,23,3.000,163.495,161.504,12.000,5,3.000,1977,0.014,B,B,2,475,0,nan,0,0,B
7738,87,1.000,39.710,19.539,8.000,4,17.000,1986,0.100,B,B,43,7227,0,nan,1,6,A
1579,48,3.000,96.057,98.153,1.000,15,1.000,2017,0.041,B,B,46,9515,5,nan,1,10,B
5058,77,3.000,79.196,44.421,10.000,16,17.000,1984,0.298,B,B,16,4048,3,nan,1,3,B


In [66]:
y_train

9069    88,504.385
2603   207,007.957
7738   182,126.281
1579   524,365.551
5058   322,048.434
           ...    
5734   150,136.611
5191   328,680.869
5390   255,102.407
860    182,236.049
7270   443,365.496
Name: Price, Length: 7000, dtype: float64

Преобразуем датафреймы для этого сделаем копию обучающего датафрейма, чтобы использовать ее для вычисления заменяющих характеристик

In [67]:
stencil_df = X_train.copy()

In [68]:
X_train = preliminary_preprocessing(X_train, stencil_df)
X_train = preliminary_preprocessing_2(X_train, stencil_df)

Посмотрим что получилось

In [69]:
X_train.describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear,Is_first_floor,Is_last_floor,HouseFloor,Ecology_1,Social_1,Social_3,Helthcare_2,Shops_1
count,"7,000.000","7,000.000","7,000.000","7,000.000","7,000.000","7,000.000","7,000.000","7,000.000","7,000.000","7,000.000","7,000.000","7,000.000","7,000.000","7,000.000"
mean,0.416,1.887,56.251,35.128,5.677,35.084,0.023,0.089,12.655,0.118,24.768,0.174,1.313,0.206
std,0.493,0.809,19.057,14.901,3.912,18.307,0.149,0.284,6.851,0.119,17.584,0.379,1.488,0.405
min,0.000,1.000,25.000,6.904,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,1.000,41.745,24.881,1.000,19.000,0.000,0.000,9.000,0.018,6.000,0.000,0.000,0.000
50%,0.000,2.000,52.634,32.895,6.000,43.000,0.000,0.000,14.000,0.075,25.000,0.000,1.000,0.000
75%,1.000,2.000,65.981,43.160,9.000,46.000,0.000,0.000,17.000,0.194,36.000,0.000,2.000,0.000
max,1.000,5.000,200.000,100.000,20.000,108.000,1.000,1.000,117.000,0.522,74.000,1.000,6.000,1.000


In [70]:
X_train.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear,Is_first_floor,Is_last_floor,HouseFloor,Ecology_1,Social_1,Social_3,Helthcare_2,Shops_1
9069,0,1.000,41.681,22.796,8.000,5,0,0,17.000,0.076,6,0,0,0
2603,0,3.000,163.495,100.000,12.000,43,0,0,3.000,0.014,2,0,0,0
7738,1,1.000,39.710,19.539,8.000,34,0,0,17.000,0.100,43,0,1,0
1579,0,3.000,96.057,79.937,1.000,3,0,0,1.000,0.041,46,0,1,1
5058,1,3.000,79.196,44.421,10.000,36,0,0,17.000,0.298,16,0,1,0


Отформатируем тестовый датасет

In [71]:
X_test = preliminary_preprocessing(X_test, stencil_df)
X_test = preliminary_preprocessing_2(X_test, stencil_df)

Посмотрим на рузельтат

In [72]:
X_test.describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear,Is_first_floor,Is_last_floor,HouseFloor,Ecology_1,Social_1,Social_3,Helthcare_2,Shops_1
count,"3,000.000","3,000.000","3,000.000","3,000.000","3,000.000","3,000.000","3,000.000","3,000.000","3,000.000","3,000.000","3,000.000","3,000.000","3,000.000","3,000.000"
mean,0.405,1.893,56.204,35.218,5.592,35.258,0.026,0.095,12.504,0.121,24.499,0.169,1.334,0.196
std,0.491,0.816,19.386,15.125,3.831,18.650,0.158,0.293,6.597,0.120,17.414,0.375,1.506,0.397
min,0.000,1.000,25.000,10.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,1.000,41.836,24.965,1.000,18.000,0.000,0.000,9.000,0.020,6.000,0.000,0.000,0.000
50%,0.000,2.000,52.112,32.554,6.000,43.000,0.000,0.000,12.000,0.076,25.000,0.000,1.000,0.000
75%,1.000,2.000,65.762,43.836,9.000,47.000,0.000,0.000,17.000,0.196,35.000,0.000,3.000,0.000
max,1.000,5.000,200.000,100.000,20.000,110.000,1.000,1.000,48.000,0.522,74.000,1.000,6.000,1.000


In [73]:
X_test.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear,Is_first_floor,Is_last_floor,HouseFloor,Ecology_1,Social_1,Social_3,Helthcare_2,Shops_1
6252,0,2.000,48.812,34.680,5.000,57,0,1,5.000,0.034,25,0,0,0
4684,1,3.000,81.103,49.310,8.000,60,0,1,4.000,0.298,16,0,1,0
1731,0,3.000,82.883,10.000,1.000,43,0,0,17.000,0.243,5,0,0,0
4742,0,1.000,33.863,29.993,0.000,43,0,0,4.000,0.035,0,0,0,0
4521,1,1.000,43.095,25.274,1.000,43,0,0,17.000,0.371,34,0,2,0


Обучим модель на основе линейной регресии. Однако для этого нужно сделать небольшое преобразование. Бинарные признаки по ранговым переменным имеют градацию от 0 до 1. Это может создать проблему, поскольку умножение коэффициента на ноль приведет к его обнулению и возможно не будут добыты определенные закономерности. Поэтому напишем функцию, которая ко всем наблюдениям прибавляет единицу и датафрейм обернутый в эту функцию будем сразу отправлять в модель

In [74]:
def add_1(df):
    output = df.copy()
    for column in output.columns:
        output[column] += 1
    return output.to_numpy()

In [75]:
from sklearn.linear_model import Lasso

In [76]:
linear = Lasso(alpha=1).fit(add_1(X_train), y_train.to_numpy())

In [77]:
linear.score(add_1(X_test), y_test.to_numpy())

0.49530984599076194

Увеличение штрафного коэффициента не дает значимых результатов, для линейной регрессии это видимо наилучшая модель. Построим модель случайного леса

In [78]:
from sklearn.ensemble import RandomForestRegressor

In [79]:
forest = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42).fit(X_train.to_numpy(), y_train.to_numpy())

In [80]:
forest.score(X_test.to_numpy(), y_test.to_numpy())

0.7024445407492923

Дальнейшие изменения коэффициентов не дают лучшего результата. Остановимся на этой модели для случайного леса. Дальше рассмотрим модель опорных векторов для регрессии. Однако для входа на эту модель признаки должны подаваться желательно в одинаковом масштабе, иначе расстояния между векторами не будут одинаковыми. Поэтому сделаем функцию, которая используя MinMaxScaler масштабирует непрерыные, а ранговые бинарные убирает

In [81]:
def svr_input(df, df_for_calc):
    output = df.copy()
    output.drop(columns=["DistrictId", "Social_3", "Is_first_floor", "Is_last_floor"], inplace=True)
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler().fit(df_for_calc)
    output = scaler.transform(df)
    return output

In [82]:
from sklearn.svm import SVR

In [83]:
stencil_df_svr = X_train.copy()

In [84]:
supp_vector = SVR(kernel="rbf", C=10, gamma=0.1).fit(svr_input(X_train,stencil_df_svr), y_train.to_numpy())

In [85]:
supp_vector.score(svr_input(X_test,stencil_df_svr), y_test.to_numpy())

-0.04002581121999271

Мда, на этих данных вообще не работает, либо я что-то неправильно делаю)). Последней моделью попробую перцептрон. Ели он не будет давать результатов выше 71%, то тогда в любом случае надо использовать случайный лес. Перцептрон так же чувствителен к в ходящим данным, поэтому сделаем функцию, которая будет возращать датафрейм по всем фичам измененный как ln(x + 1)

In [86]:
def mlr_input(df):
    output = df.copy()
    for column in output.columns:
        output[column] = np.log(output[column] + 1)
    return output.to_numpy()

In [87]:
from sklearn.neural_network import MLPRegressor

In [88]:
perceptron = MLPRegressor(hidden_layer_sizes=(5,10), activation="relu", solver="lbfgs", alpha=5, random_state=42).\
    fit(mlr_input(X_train), y_train.to_numpy())

In [89]:
perceptron.score(mlr_input(X_test), y_test.to_numpy())

0.5260901874928421

В итоге модель тоже не имеет достаточной предскзаательной силы. В итоге выбираем модель случайного леса

In [90]:
test_df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.000,49.883,33.433,6.000,6,14.000,1972,0.310,B,B,11,2748,1,nan,0,0,B
1,15856,74,2.000,69.263,nan,1.000,6,1.000,1977,0.076,B,B,6,1437,3,nan,0,2,B
2,5480,190,1.000,13.598,15.948,12.000,2,5.000,1909,0.000,B,B,30,7538,87,"4,702.000",5,5,B
3,15664,47,2.000,73.047,51.941,9.000,22,22.000,2007,0.102,B,B,23,4583,3,nan,3,3,B
4,14275,27,1.000,47.527,43.388,1.000,17,17.000,2017,0.072,B,B,2,629,1,nan,0,0,A


In [91]:
test_df.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,"5,000.000","5,000.000","5,000.000","5,000.000","3,959.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","2,623.000","5,000.000","5,000.000"
mean,"8,412.595",51.279,1.910,56.450,36.159,5.977,8.632,12.601,"1,984.393",0.120,24.934,"5,406.900",8.263,"1,146.657",1.319,4.243
std,"4,832.674",44.179,0.839,19.093,17.825,9.950,5.483,6.789,18.573,0.120,17.532,"4,026.615",23.864,"1,044.744",1.480,4.777
min,1.000,0.000,0.000,1.379,0.333,0.000,1.000,0.000,"1,908.000",0.000,0.000,168.000,0.000,0.000,0.000,0.000
25%,"4,221.750",21.000,1.000,41.906,23.092,1.000,4.000,9.000,"1,973.000",0.020,6.000,"1,564.000",0.000,325.000,0.000,1.000
50%,"8,320.500",37.000,2.000,52.921,32.925,6.000,7.000,12.000,"1,977.000",0.072,25.000,"5,285.000",2.000,900.000,1.000,3.000
75%,"12,598.250",77.000,2.000,66.285,45.174,9.000,12.000,17.000,"2,000.000",0.196,36.000,"7,287.000",5.000,"1,548.000",2.000,6.000
max,"16,795.000",212.000,17.000,223.454,303.071,620.000,78.000,99.000,"2,020.000",0.522,74.000,"19,083.000",141.000,"4,849.000",6.000,23.000


In [92]:
X_test_final = preliminary_preprocessing(test_df, stencil_df)
X_test_final = preliminary_preprocessing_2(X_test_final, stencil_df)

In [93]:
X_test_final.describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,HouseYear,Is_first_floor,Is_last_floor,HouseFloor,Ecology_1,Social_1,Social_3,Helthcare_2,Shops_1
count,"5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000","5,000.000"
mean,0.424,1.906,56.492,35.247,5.675,35.580,0.021,0.101,12.601,0.120,24.934,0.182,1.319,0.204
std,0.494,0.806,18.912,14.796,3.816,18.512,0.145,0.302,6.789,0.120,17.532,0.386,1.480,0.403
min,0.000,1.000,25.000,10.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,1.000,41.906,25.274,1.000,20.000,0.000,0.000,9.000,0.020,6.000,0.000,0.000,0.000
50%,0.000,2.000,52.921,33.156,6.000,43.000,0.000,0.000,12.000,0.072,25.000,0.000,1.000,0.000
75%,1.000,2.000,66.285,43.341,9.000,47.000,0.000,0.000,17.000,0.196,36.000,0.000,2.000,0.000
max,1.000,5.000,200.000,100.000,20.000,110.000,1.000,1.000,99.000,0.522,74.000,1.000,6.000,1.000


In [94]:
y_predict = forest.predict(X_test_final.to_numpy())

In [95]:
y_predict

array([158798.25915116, 221248.63773019, 192102.06945482, ...,
       324438.04046909, 211389.56356325, 179937.69215913])

In [96]:
y_predict = pd.DataFrame(y_predict, columns=["Price"])

In [97]:
final_output = pd.concat([test_df["Id"], y_predict], axis=1)

In [98]:
final_output.to_csv("output_csv.csv", index=False)